In [ ]:
!pip install selenium
!apt-get update
!apt-get install chromium chromium-driver
!pip install webdriver_manager
def web_driver():
    from selenium import webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver
!pip install fake_useragent



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,756 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://ppa.launchpadcontent.net/

2mars

In [ ]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from concurrent.futures import ThreadPoolExecutor

def web_driver():
    ua = UserAgent()
    user_agent = ua.random
    options = webdriver.ChromeOptions()
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

def scrape_text_from_link(driver, link):
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    data = {'SalleDeBain': None, 'surface': None, 'chambres': None, 'price': None }
    if 'immoneuf' not in link:
        divs = soup.find_all("div", class_="sc-6p5md9-1 ksqQXN")

        for div in divs:
            text = div.get_text()
            if "SalleDeBain" in text:
                data['SalleDeBain'] = int(re.search(r'\d+', text).group())
            elif "SurfaceTotale" in text:
                data['surface'] = int(re.search(r'\d+', text).group())
            elif "Chambres" in text:
                data['chambres'] = int(re.search(r'\d+', text).group())
        salon_lis = soup.find_all("li", class_="sc-qmn92k-1 jJjeGO")
        for li in salon_lis:
            if "Salons" in li.get_text():
                data['salons'] = int(re.search(r'\d+', li.get_text()).group())
        special_div = soup.find("div", class_="sc-1g3sn3w-10 leGvyq")
        if special_div:
            price_text = special_div.get_text()
            price_digits = re.sub(r'\D', '', price_text)
            if price_digits:
                data['price'] = int(price_digits)
    return data

url_base = "https://www.avito.ma/fr/casablanca/2_mars/appartements-%C3%A0_vendre?price=500000-&size=35-&spare_rooms=1-7&bathrooms=1-7&has_price=true&has_image=true&o={}"

def scrape_links(driver, url):
    driver.get(url)
    try:
        elements = WebDriverWait(driver, 3).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.sc-1jge648-0.eTbzNs"))
        )
        hrefs = [element.get_attribute("href") for element in elements]
    except Exception as e:
        print(f"Error: {e}")
        return []

    data_list = []
    for href in hrefs:
        if 'immoneuf' not in href:
            print("Scraping:", href)
            data = scrape_text_from_link(driver, href)
            data_list.append(data)
    return data_list

driver = web_driver()

links_data = []

for page_number in range(1, 6):
    url = url_base.format(page_number)
    print("Scraping page", page_number)
    data = scrape_links(driver, url)
    links_data.extend(data)

driver.quit()

df = pd.DataFrame(links_data)
print(df)
df.to_csv('avito_2mars.csv', index=False)


Scraping page 1
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_%C3%A0__2_mars_53513973.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_86_m%C2%B2_%C3%A0_Casablanca_54493039.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_92_m%C2%B2_%C3%A0_Casablanca_54760717.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_145_m%C2%B2_%C3%A0_Casablanca_54542204.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_standing_178_m%C2%B2_%C3%A0_2_mars_54879189.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_128_m%C2%B2_%C3%A0_Casablanca_54879142.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/lux_appartement_%C3%A0_2_mars__54383198.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_47_m%C2%B2_%C3%A0_Casablanca_54805136.htm
Scraping: https://www.avito.ma/fr/2_mars/appartement

In [ ]:
df

,SalleDeBain,surface,chambres,price,salons
0,2,144,3,1650000,2
1,2,98,3,1680000,2
2,2,92,2,1240000,2
3,2,144,3,2150000,1
4,3,190,3,2470000,2
...,...,...,...,...,...
136,3,213,4,2348000,1
137,2,170,3,2000000,2
138,1,71,3,760000,1
139,3,121,3,1700000,1


In [ ]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from concurrent.futures import ThreadPoolExecutor

def web_driver():
    ua = UserAgent()
    user_agent = ua.random
    options = webdriver.ChromeOptions()
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

def scrape_text_from_link(driver, link):
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    data = {'SalleDeBain': None, 'surface': None, 'chambres': None, 'salons': None,'Secteur': None ,  'price': None  }
    if 'immoneuf' not in link:
        divs = soup.find_all("div", class_="sc-6p5md9-1 ksqQXN")

        for div in divs:
            text = div.get_text()
            if "SalleDeBain" in text:
                data['SalleDeBain'] = int(re.search(r'\d+', text).group())
            elif "SurfaceTotale" in text:
                data['surface'] = int(re.search(r'\d+', text).group())
            elif "Chambres" in text:
                data['chambres'] = int(re.search(r'\d+', text).group())


        salon_lis = soup.find_all("li", class_="sc-qmn92k-1 jJjeGO")
        for li in salon_lis:
            if "Salons" in li.get_text():
                data['salons'] = int(re.search(r'\d+', li.get_text()).group())
            if "Secteur" in li.get_text():
                sector_name = re.sub(r'^Secteur\s*', '', li.get_text())
                data['Secteur'] = sector_name.strip()
        special_div = soup.find("div", class_="sc-1g3sn3w-10 leGvyq")
        if special_div:
            price_text = special_div.get_text()
            price_digits = re.sub(r'\D', '', price_text)
            if price_digits:
                data['price'] = int(price_digits)
    return data

url_base = "https://www.avito.ma/fr/casablanca/2_mars/appartements-%C3%A0_vendre?price=500000-&size=35-&spare_rooms=1-7&bathrooms=1-7&has_price=true&has_image=true&o={}"

def scrape_links(driver, url):
    driver.get(url)
    try:
        elements = WebDriverWait(driver, 3).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.sc-1jge648-0.eTbzNs"))
        )
        hrefs = [element.get_attribute("href") for element in elements]
    except Exception as e:
        print(f"Error: {e}")
        return []

    data_list = []
    for href in hrefs:
        if 'immoneuf' not in href:
            print("Scraping:", href)
            data = scrape_text_from_link(driver, href)
            data_list.append(data)
    return data_list

driver = web_driver()

links_data = []

for page_number in range(1, 2):
    url = url_base.format(page_number)
    print("Scraping page", page_number)
    data = scrape_links(driver, url)
    links_data.extend(data)

driver.quit()

df = pd.DataFrame(links_data)
print(df)
df.to_csv('avito_2mars.csv', index=False)


Scraping page 1
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_145_m%C2%B2_%C3%A0_Casablanca_54542204.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_%C3%A0__2_mars_53513973.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_86_m%C2%B2_%C3%A0_Casablanca_54493039.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_92_m%C2%B2_%C3%A0_Casablanca_54760717.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_standing_178_m%C2%B2_%C3%A0_2_mars_54879189.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_128_m%C2%B2_%C3%A0_Casablanca_54879142.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/lux_appartement_%C3%A0_2_mars__54383198.htm
Scraping: https://www.avito.ma/fr/2_mars/appartements/Appartement_%C3%A0_vendre_47_m%C2%B2_%C3%A0_Casablanca_54805136.htm
Scraping: https://www.avito.ma/fr/2_mars/appartement

ALL DATA

In [14]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from concurrent.futures import ThreadPoolExecutor
from google.colab import files

def web_driver():
    ua = UserAgent()
    user_agent = ua.random
    options = webdriver.ChromeOptions()
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

def scrape_text_from_link(driver, link):
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    data = {'SalleDeBain': None, 'surface': None, 'chambres': None, 'salons': None,'Secteur': None ,  'price': None  }
    if 'immoneuf' not in link:
        divs = soup.find_all("div", class_="sc-6p5md9-1 ksqQXN")

        for div in divs:
            text = div.get_text()
            if "SalleDeBain" in text:
                data['SalleDeBain'] = int(re.search(r'\d+', text).group())
            elif "SurfaceTotale" in text:
                data['surface'] = int(re.search(r'\d+', text).group())
            elif "Chambres" in text:
                data['chambres'] = int(re.search(r'\d+', text).group())


        salon_lis = soup.find_all("li", class_="sc-qmn92k-1 jJjeGO")
        for li in salon_lis:
            if "Salons" in li.get_text():
                data['salons'] = int(re.search(r'\d+', li.get_text()).group())
            if "Secteur" in li.get_text():
                sector_name = re.sub(r'^Secteur\s*', '', li.get_text())
                data['Secteur'] = sector_name.strip()
        special_div = soup.find("div", class_="sc-1g3sn3w-10 leGvyq")
        if special_div:
            price_text = special_div.get_text()
            price_digits = re.sub(r'\D', '', price_text)
            if price_digits:
                data['price'] = int(price_digits)
    return data
url_base = "https://www.avito.ma/fr/casablanca/appartements-%C3%A0_vendre?rooms=1-11&size=40-300&spare_rooms=1-7&bathrooms=1-7&price=400000-3000000&o={}"

def scrape_links(driver, url):
    driver.get(url)
    try:
        elements = WebDriverWait(driver, 3).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.sc-1jge648-0.eTbzNs"))
        )
        hrefs = [element.get_attribute("href") for element in elements]
    except Exception as e:
        print(f"Error: {e}")
        return []

    data_list = []
    for href in hrefs:
        if 'immoneuf' not in href:
            print("Scraping:", href)
            data = scrape_text_from_link(driver, href)
            data_list.append(data)
    return data_list

driver = web_driver()

links_data = []

for page_number in range(1, 146):
    url = url_base.format(page_number)
    print("Scraping page", page_number)
    data = scrape_links(driver, url)
    links_data.extend(data)

driver.quit()

df = pd.DataFrame(links_data)
print(df)
df.to_csv('avito1.csv', index=False, encoding='utf-8-sig')
files.download('avito1.csv')


Streaming output truncated to the last 5000 lines.
Scraping: https://www.avito.ma/fr/sidi_bernoussi/appartements/%D8%B4%D9%82%D8%A9_%D9%84%D9%84%D8%A8%D9%8A%D8%B9_%D8%A8%D8%A5%D9%82%D8%A7%D9%85%D8%A9_%D8%A7%D9%84%D8%A3%D9%85%D8%A7%D9%86%D8%A9_%D8%B3%D9%8A_%D9%8A_%D8%A7%D9%84%D8%A8%D8%B1%D9%86%D9%88%D8%B5%D9%8A_54792311.htm
Scraping: https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_%C3%A0_vendre_80_m%C2%B2_%C3%A0_Casablanca_54792067.htm
Scraping: https://www.avito.ma/fr/palmier/appartements/Appartement_%C3%A0_vendre_93_m%C2%B2_%C3%A0_Casablanca_54661149.htm
Scraping: https://www.avito.ma/fr/autre_secteur/appartements/Les_Perles_de_Tamaris_Appartement_R%C3%A9nov%C3%A9__54791711.htm
Scraping: https://www.avito.ma/fr/a%C3%AFn_borja/appartements/Appartement_%C3%A0_vendre_64_m%C2%B2_%C3%A0_Casablanca_54791062.htm
Scraping: https://www.avito.ma/fr/abdelmoumen/appartements/Superbe_Appartement__%C3%A0_vendre__54790986.htm
Scraping: https://www.avito.ma/fr/maarif/appartements/Appa

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')